# Visualization of Torsiondrive 2D scan result as energy countour and interactively showing molecular structures.


## Specify your scan.xyz (produced by torsiondrive) and scan.pdb (converted from scan.xyz by Molecule.py) below.

In [1]:
scanxyz = "/home/winter/onsager/tsqm/td/56far/scan.xyz"
scanpdb = "/home/winter/onsager/tsqm/td/56far/scan.pdb"

## Execute the following cell to see an interactive plot. Click on the scan point to show corresponding structure.

In [4]:
import numpy as np
import nglview as nv
from IPython.display import display
import plotly.graph_objs as go
import plotly

def show_structure_for_click(trace, point, selector):
    x=point.xs[0]
    y=point.ys[0]
    print ("Showing optimized conformation for x,y = " +str(x)+","+str(y))
    # pick the lines correponding to (x,y) in scan.pdb and write a temporary pdb for visualization.
    position = int(1 + (y-ymin)//ystep + (x-xmin)//xstep * ypts)
    with open(scanpdb) as f:
        lines = f.readlines()
    wanted_lines = lines[1+(n_atoms+3)*(position-1):1+(n_atoms+3)*position]
    with open('temp.pdb', 'w') as f:
        for item in wanted_lines:
            f.write("%s" % item)
    view = nv.show_structure_file("temp.pdb")
    view.background = 'white'
    display(view)

# Parse dihedrals and energies. From Yudong's script.
with open(scanxyz) as f:
    lines = f.readlines()
global n_atoms 
n_atoms = int(lines[0])
comment_lines = lines[1::n_atoms+2]
grid_data = dict()
for line in comment_lines:
    ls = line.strip().split()
    assert ls[0] == 'Dihedral' and ls[-2] == 'Energy', line
    grid_energy = float(ls[-1])
    grid_coord = []
    for i in range(1, len(ls) - 2):
        c = int(ls[i].replace('(', '').replace(',','').replace(')',''))
        grid_coord.append(c)
    grid_data[tuple(grid_coord)] = grid_energy

# auto determination of No. of scan points (xpts, ypts), minimum values (xmin, ymin) and step size (xstep,ystep)
i = 0
while lines[1+i*(n_atoms+2)].strip().split()[1] == lines[1+(i+1)*(n_atoms+2)].strip().split()[1] != 0:
    i=i+1
global xpts, ypts, xmin, ymin, xstep, ystep
ypts = i+1
xpts = len(grid_data) // ypts
xmin = float(lines[1].strip().split()[1][1:-1])
ymin = float(lines[1].strip().split()[2][0:-1])
xstep = float(lines[1+ypts*(n_atoms+2)].strip().split()[1][1:-1]) - float(lines[1].strip().split()[1][1:-1])
ystep = float(lines[1+(n_atoms+2)].strip().split()[2][0:-1]) - float(lines[1].strip().split()[2][0:-1])

x = np.linspace(xmin,xmin+xstep*(xpts-1),num=xpts)
y = np.linspace(ymin,ymin+ystep*(ypts-1),num=ypts)
rawe = np.reshape(np.fromiter(grid_data.values(), dtype=float),(xpts,ypts)).T
# convering rawe in hartree to e in kcal/mol with lowest value offset to zero.
e = rawe*627.509-np.min(rawe*627.509)

# Make the plot.  
fig = go.FigureWidget([go.Contour(x=x, y=y, z=e, 
                            colorscale='YlGnBu', ncontours=12,
                            colorbar=dict(nticks=12,title='Energy (kcal/mol)')
                           )])
#There is a glitch rendering figure widget if go.Layout(...) is added to go.FigureWidget(). 
#For custom layout, use the cell below that renders static figure.

fig.data[0].on_click(show_structure_for_click)
fig

FigureWidget({
    'data': [{'colorbar': {'nticks': 12, 'title': {'text': 'Energy (kcal/mol)'}},
             …

## Too many structures? Re-execute the above cell to clear them.

## Wanna see a static energy surface? Execute the cell below.

In [3]:
plotly.offline.init_notebook_mode(connected=True)
static = go.Figure([go.Contour(x=x, y=y, z=e, 
                            colorscale='YlGnBu', ncontours=12,
                            colorbar=dict(nticks=12,title='Energy (kcal/mol)')
                           )],
                   go.Layout(width=xpts*xstep*8, height=ypts*ystep*8+105,
                          xaxis=dict(tickvals=x, tickangle=30),
                          yaxis=dict(tickvals=y)))
plotly.offline.iplot(static)

## Known issues: this notebook doesn't work if opened in jupyterlab, so open it by vanilla jupyter notebook.